# Project Final Report

Group Member: Mia , Selima, Daniel 

## (1) Introduction

The project that we are working on surrounds the prediction of the usage of a video game research server. We are provided two datasets as a supplement to answer any predictive questions we came up with. This dataset is provided by the Pacific Laboratory for Artificial Intelligence, and they aim to understand in more detail the actions of players in the video game, Minecraft.


>**Question:** How does the amount of hours a player spend on the server, and the number of sessions can accurately predict gender?

- The number of sessions and the amount of hours are the predictor variables
- The gender of players is the response variable of interest


## (2) Methods & Results

We first load the data using HTML form, which allows us to see the dataset in the CSV format. Then, we select the variables in question: hashedEmail, played_hours, and gender from the players.csv dataset, and hashedEmail, sessioncount from the second dataset. We join the two datasets together. 


In [2]:
library(tidyverse)
library(repr)
library(tidymodels)
library(dplyr)
options(repr.matrix.max.rows = 6)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.6     ✔ rsample      1.2.1
✔ dials        1.3.0     ✔ tune         1.1.2
✔ infer        1.0.7     ✔ workflows    1.1.4
✔ modeldata    1.4.0     ✔ workflowsets 1.0.1
✔ parsnip      1.2.1     ✔ yardstick    1.3.1
✔ recipes      1.1.0     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filt

In [ ]:
players <- read_csv("data/players.csv") |>
    select(hashedEmail, played_hours, gender)


sessions <- read_csv("data/sessions.csv")  |> 
    select(hashedEmail) |>
    group_by(hashedEmail) |>
    summarize(session_count = n())

data <- left_join(sessions, players, by = "hashedEmail")

data